Docker Compose 起動エラー調査用ノートブック

このノートブックは各サービスごとに1セルを割り当て、各エンドポイントへ順にリクエストを送りステータスとレスポンスを確認します。

In [ ]:
# --- ライブラリのインポート ---
import requests
import time
import traceback
import pandas as pd
from IPython.display import display
import docker
import subprocess
import logging
from pathlib import Path
from contextlib import contextmanager
from requests.exceptions import RequestException

# ロガー設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 設定
REQUEST_TIMEOUT = 5  # seconds
RETRY_COUNT = 1
RETRY_DELAY = 1

print("Imports done. REQUEST_TIMEOUT=", REQUEST_TIMEOUT)

In [ ]:
# --- SERVICES 定義 ---
SERVICES = {
    "1-coordinator-ring": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "2-quorum-consistency": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "3-sharding-replica": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "4-distributed-lock": {
        "port": 8000,
        "endpoints": {
            "health": "/stats",
            "acquire": "/acquire",
            "release": "/release"
        },
        "test_data": {"resource": "test_resource", "owner": "test_client"}
    },
    "5-cache-aside": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "get": "/get",
            "set": "/set"
        },
        "test_data": {"entity_type": "user", "entity_id": "1", "data": {"name": "test"}}
    },
    "6-bloom-sstable": {
        "port": 5000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "7-rate-limiting": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "check": "/check_rate"
        },
        "test_data": {"user_id": "test_user"}
    },
    "8-line-streams": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "produce": "/produce",
            "consume": "/consume"
        },
        "test_data": {"message": "test_message", "consumer": "test_consumer"}
    },
    "9-session-store": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "login": "/login",
            "me": "/me"
        },
        "test_data": {"username": "test_user"}
    },
    "10-leaderboard": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "score": "/score",
            "top": "/top/10"
        },
        "test_data": {"user_id": "test_user", "score": 100}
    }
}

print("SERVICES configured with", len(SERVICES), "services")

In [ ]:
# --- Docker管理クラス定義 ---
# パス設定
BASE_DIR = Path("/Users/codefox/workspace/practice_infra_arch/key_value_store")

class DockerManager:
    """Dockerコンテナの管理クラス"""
    
    def __init__(self, service_name):
        self.service_name = service_name
        self.service_dir = BASE_DIR / service_name
        self.client = docker.from_env()
        
    def start_service(self):
        """サービスを起動"""
        logger.info(f"🚀 {self.service_name} サービスを起動中...")
        try:
            cmd = ["docker", "compose", "up", "-d", "--build"]
            result = subprocess.run(
                cmd, 
                cwd=self.service_dir, 
                capture_output=True, 
                text=True,
                timeout=300  # 5分でタイムアウト
            )
            
            if result.returncode != 0:
                logger.error(f"❌ {self.service_name} 起動失敗: {result.stderr}")
                return False
                
            logger.info(f"✅ {self.service_name} サービス起動完了")
            return True
            
        except subprocess.TimeoutExpired:
            logger.error(f"❌ {self.service_name} 起動タイムアウト")
            return False
        except Exception as e:
            logger.error(f"❌ {self.service_name} 起動中にエラー: {str(e)}")
            return False
    
    def stop_service(self):
        """サービスを停止"""
        logger.info(f"🛑 {self.service_name} サービスを停止中...")
        try:
            cmd = ["docker", "compose", "down", "-v"]
            result = subprocess.run(
                cmd, 
                cwd=self.service_dir, 
                capture_output=True, 
                text=True,
                timeout=60
            )
            
            if result.returncode == 0:
                logger.info(f"✅ {self.service_name} サービス停止完了")
            else:
                logger.warning(f"⚠️ {self.service_name} 停止時に警告: {result.stderr}")
                
        except Exception as e:
            logger.error(f"❌ {self.service_name} 停止中にエラー: {str(e)}")
    
    def health_check(self, max_retries=30):
        """ヘルスチェック"""
        service_config = SERVICES[self.service_name]
        health_url = f"http://localhost:{service_config['port']}{service_config['endpoints']['health']}"
        
        for attempt in range(max_retries):
            try:
                response = requests.get(health_url, timeout=5)
                if response.status_code == 200:
                    logger.info(f"✅ {self.service_name} ヘルスチェック成功")
                    return True
            except requests.exceptions.RequestException:
                pass
                
            time.sleep(2)
            logger.info(f"⏳ {self.service_name} ヘルスチェック待機中... ({attempt + 1}/{max_retries})")
        
        logger.error(f"❌ {self.service_name} ヘルスチェック失敗")
        return False

@contextmanager
def service_context(service_name):
    """サービスのライフサイクル管理コンテキストマネージャー"""
    docker_manager = DockerManager(service_name)
    docker_manager.stop_service()
    
    try:
        # サービス起動
        if not docker_manager.start_service():
            raise RuntimeError(f"Failed to start {service_name}")
        
        # ヘルスチェック
        if not docker_manager.health_check():
            raise RuntimeError(f"Health check failed for {service_name}")
        
        yield docker_manager
        
    finally:
        # サービス停止
        docker_manager.stop_service()
        # クールタイム: コンテナの完全解放 / ポート解放待ち
        logger.info(f"⏳ {service_name} 停止後のクールタイム: 30秒待機します...")
        time.sleep(30)

print("✅ Docker管理クラスが定義されました")

In [ ]:
# --- ヘルパー関数: send_request ---

def try_parse_json(resp):
    try:
        return resp.json()
    except Exception:
        return None


def send_request(service_name, path, method='GET', json_body=None, params=None, timeout=REQUEST_TIMEOUT, session=None):
    cfg = SERVICES.get(service_name)
    port = cfg.get('port') if cfg else None
    url = f"http://localhost:{port}{path}"
    method = method.upper()
    last_exc = None

    for attempt in range(1, RETRY_COUNT+1):
        try:
            start = time.time()
            if session is not None:
                resp = session.request(method, url, json=json_body, params=params, timeout=timeout)
            else:
                resp = requests.request(method, url, json=json_body, params=params, timeout=timeout)
            elapsed = time.time() - start

            parsed = try_parse_json(resp)
            body = parsed if parsed is not None else resp.text

            return {
                'service': service_name,
                'url': url,
                'method': method,
                'status_code': resp.status_code,
                'body': body,
                'elapsed': elapsed,
                'error': None
            }
        except RequestException as e:
            last_exc = e
            time.sleep(RETRY_DELAY)
        except Exception as e:
            last_exc = e
            time.sleep(RETRY_DELAY)

    # all retries failed
    return {
        'service': service_name,
        'url': url,
        'method': method,
        'status_code': None,
        'body': None,
        'elapsed': None,
        'error': ''.join(traceback.format_exception_only(type(last_exc), last_exc)).strip()
    }

print('Helper functions defined')

In [ ]:
# --- 1-coordinator-ring のテスト ---
service = '1-coordinator-ring'
cfg = SERVICES[service]
print('Testing', service)

with service_context(service):
    # health
    r = send_request(service, cfg['endpoints']['health'], method='GET')
    print(r)

    # write
    r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
    print(r_write)

    # read (use params)
    params = {'key': cfg['test_data'].get('key')}
    r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
    print(r_read)

In [ ]:
# --- 2-quorum-consistency のテスト ---
service = '2-quorum-consistency'
cfg = SERVICES[service]
print('Testing', service)

with service_context(service):
    r = send_request(service, cfg['endpoints']['health'], method='GET')
    print(r)

    r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
    print(r_write)

    time.sleep(0.5)
    params = {'key': cfg['test_data'].get('key')}
    r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
    print(r_read)

In [ ]:
# --- 3-sharding-replica のテスト ---
service = '3-sharding-replica'
cfg = SERVICES[service]
print('Testing', service)

with service_context(service):
    try:
        r = send_request(service, cfg['endpoints']['health'], method='GET')
        print(r)

        r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
        print(r_write)

        params = {'key': cfg['test_data'].get('key')}
        r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
        print(r_read)
    except Exception as e:
        print('Exception during test:', e)
        traceback.print_exc()

In [ ]:
# --- 4-distributed-lock のテスト ---
service = '4-distributed-lock'
cfg = SERVICES[service]
print('Testing', service)

with service_context(service):
    r = send_request(service, cfg['endpoints']['health'], method='GET')
    print('health:', r)

    r_acq = send_request(service, cfg['endpoints']['acquire'], method='POST', json_body=cfg['test_data'])
    print('acquire:', r_acq)

    r_rel = send_request(service, cfg['endpoints']['release'], method='POST', json_body=cfg['test_data'])
    print('release:', r_rel)

In [ ]:
# --- 5-cache-aside のテスト ---
service = '5-cache-aside'
cfg = SERVICES[service]
print('Testing', service)

with service_context(service):
    r = send_request(service, cfg['endpoints']['health'], method='GET')
    print('health:', r)

    # set
    r_set = send_request(service, cfg['endpoints']['set'], method='POST', json_body=cfg['test_data'])
    print('set:', r_set)

    # get
    params = {'entity_type': cfg['test_data']['entity_type'], 'entity_id': cfg['test_data']['entity_id']}
    r_get = send_request(service, cfg['endpoints']['get'], method='GET', params=params)
    print('get:', r_get)

    # basic assertion
    if r_get.get('body') is not None:
        if isinstance(r_get['body'], dict) and r_get['body'].get('data') == cfg['test_data']['data']:
            print('GET returned expected data')
        else:
            print('GET did not return expected data:', r_get['body'])

In [ ]:
# --- 6-bloom-sstable のテスト ---
service = '6-bloom-sstable'
cfg = SERVICES[service]
print('Testing', service)

with service_context(service):
    r = send_request(service, cfg['endpoints']['health'], method='GET')
    print('health:', r)

    r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
    print('write:', r_write)

    params = {'key': cfg['test_data'].get('key')}
    r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
    print('read:', r_read)

In [ ]:
# --- 7-rate-limiting のテスト ---
service = '7-rate-limiting'
cfg = SERVICES[service]
print('Testing', service)

with service_context(service):
    r = send_request(service, cfg['endpoints']['health'], method='GET')
    print('health:', r)

    # call check_rate multiple times
    results = []
    for i in range(5):
        r_check = send_request(service, cfg['endpoints']['check'], method='GET', params=cfg['test_data'])
        print(f'check attempt {i+1}:', r_check)
        results.append(r_check)
        time.sleep(0.2)

In [ ]:
# --- 8-line-streams のテスト ---
service = '8-line-streams'
cfg = SERVICES[service]
print('Testing', service)

with service_context(service):
    r = send_request(service, cfg['endpoints']['health'], method='GET')
    print('health:', r)

    # produce
    r_prod = send_request(service, cfg['endpoints']['produce'], method='POST', json_body=cfg['test_data'])
    print('produce:', r_prod)

    # consume
    r_consume = send_request(service, cfg['endpoints']['consume'], method='GET', params={'consumer': cfg['test_data'].get('consumer')})
    print('consume:', r_consume)

In [ ]:
# --- 9-session-store のテスト ---
service = '9-session-store'
cfg = SERVICES[service]
print('Testing', service)

with service_context(service):
    session = requests.Session()

    r = send_request(service, cfg['endpoints']['health'], method='GET', session=session)
    print('health:', r)

    # login
    r_login = send_request(service, cfg['endpoints']['login'], method='POST', json_body=cfg['test_data'], session=session)
    print('login:', r_login)

    # me
    r_me = send_request(service, cfg['endpoints']['me'], method='GET', session=session)
    print('me:', r_me)

In [ ]:
# --- 10-leaderboard のテスト ---
service = '10-leaderboard'
cfg = SERVICES[service]
print('Testing', service)

with service_context(service):
    r = send_request(service, cfg['endpoints']['health'], method='GET')
    print('health:', r)

    # score
    r_score = send_request(service, cfg['endpoints']['score'], method='POST', json_body=cfg['test_data'])
    print('score:', r_score)

    # top
    r_top = send_request(service, cfg['endpoints']['top'], method='GET')
    print('top:', r_top)

    # check if user in top
    if isinstance(r_top.get('body'), list):
        found = any((item.get('user_id') == cfg['test_data']['user_id']) for item in r_top['body'])
        print('User present in top:', found)

In [ ]:
# --- 全サービス一括実行とサマリ ---
results = []

for service_name in SERVICES.keys():
    cfg = SERVICES[service_name]
    print('\n=== SERVICE:', service_name, '===')

    with service_context(service_name):
        # use a session only for services that need it
        session = requests.Session() if service_name == '9-session-store' else None

        for ep_name, path in cfg['endpoints'].items():
            method = 'GET'
            if ep_name in ('write', 'set', 'acquire', 'release', 'produce', 'login', 'score'):
                method = 'POST'

            json_body = None
            params = None
            if method == 'POST':
                json_body = cfg.get('test_data')
            else:
                # construct params from common key names
                td = cfg.get('test_data', {})
                if 'key' in td:
                    params = {'key': td['key']}
                elif 'entity_type' in td and 'entity_id' in td:
                    params = {'entity_type': td['entity_type'], 'entity_id': td['entity_id']}
                elif 'consumer' in td:
                    params = {'consumer': td['consumer']}
                elif 'user_id' in td:
                    params = {'user_id': td['user_id']}

            r = send_request(service_name, path, method=method, json_body=json_body, params=params, session=session)
            results.append(r)
            print(r)

# summary
df = pd.DataFrame(results)
if 'status_code' in df.columns:
    df['success'] = df['status_code'].apply(lambda s: 200 <= s < 300 if pd.notnull(s) else False)

display(df)

failed = df[~df['success']]
if not failed.empty:
    print('\nFailures:')
    display(failed)
else:
    print('\nAll endpoints returned 2xx statuses where applicable')

実行方法:

- 必要なパッケージをインストール: pip install requests pandas docker
- Jupyter で開いて各セルを順に実行するか、一括で実行して結果を確認してください。
- docker-compose のサービスが別ポートにバインドされている場合は `SERVICES` の port を調整してください。

備考: タイムアウトや接続拒否の詳細はセル出力に表示されます。ネットワークやコンテナのログと合わせて調査してください。